<a href="https://colab.research.google.com/github/sainikhit2k/DL/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All of the architectures are from the following link:
https://towardsdatascience.com/illustrated-10-cnn-architectures-95d78ace614d#e276

<h1>Tensorflow implementation of AlexNet <h1>

In [22]:
import tensorflow as tf
from keras.layers import Input, Flatten,Dense, Conv2D, MaxPool2D, ZeroPadding2D
from keras.models import Model


In [23]:
def conv_pool_block(x, n_filters, kernel_size, stride, padd=None, pool=True):

  if padd:
    x = ZeroPadding2D(padding=(padd,padd))(x)
  x = Conv2D(n_filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
  if pool:
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

  return x


def AlexNet(input_shape=(227, 227, 3)):
  inp_img = Input(shape=input_shape)
  out = conv_pool_block(inp_img, 96, (11,11), (4,4))
  out = conv_pool_block(out, 256, (5,5), (1,1), padd=2)
  out = conv_pool_block(out, 384, (3,3),(1,1), padd=1, pool=False)
  out = conv_pool_block(out, 384, (3,3),(1,1), padd=1, pool=False)
  out = conv_pool_block(out, 256, (3,3), (1,1), padd=1, pool=False)
  out = MaxPool2D(pool_size=(3,3), strides=(2, 2))(out)
  out = Flatten()(out)
  out = Dense(4096, activation='relu')(out)
  out = Dense(4096, activation='relu')(out)
  out = Dense(1000, activation='softmax')(out)
  model = Model(inp_img, out)
  model.summary()



alexNet = AlexNet()



Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_18 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
zero_padding2d_19 (ZeroPaddi (None, 15, 15, 256)       0  

In [27]:
import torch
import torch.nn as nn
from torchsummary import summary 

In [87]:
class ConvPool(nn.Module):
  def __init__(self, in_channels, n_filters, kernel_size, stride, padd=0, pool=True):
    super(ConvPool, self).__init__()
    self.conv = nn.Conv2d(in_channels, n_filters, kernel_size, stride, padding=padd)
    self.ReLU = nn.ReLU()
    self.pool = pool
    self.max_pool = nn.MaxPool2d((3,3), stride=2)

  def forward(self, x):
    x = self.ReLU(self.conv(x)) 
    if self.pool:
      x = self.max_pool(x)
    return x


# def ConvPool(in_channels, n_filters, kernel_size, stride, padd=0, pool=True):
#   modules = []
#   modules.append(nn.Conv2d(in_channels, n_filters, kernel_size, stride, padding=padd))
#   modules.append(nn.ReLU(inplace=True))
#   if pool:
#     modules.append(nn.MaxPool2d((3,3), stride=2))
  
#   block = nn.Sequential(*modules)
#   return block

# test = ConvPool(3, 96, (11,11), 4)
# summary(test, (3, 227, 227))


In [88]:
class AlexNet(nn.Module):
  def __init__(self):
    super(AlexNet, self).__init__()
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax()
    self.conv1 = ConvPool(3, 96, (11,11), 4)
    self.conv2 = ConvPool(96, 256, (5,5), 1, 2)
    self.conv3 = nn.Conv2d(256, 384, (3,3), 1, 1)
    self.conv4 = nn.Conv2d(384, 384, (3,3), 1, 1)
    self.conv5 = ConvPool(384, 256, (3,3), 1, 1)
    self.fc1 = nn.Linear(6*6*256, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 1000)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.relu(self.conv3(x))
    x = self.relu(self.conv4(x))
    x = self.conv5(x)
    x = torch.flatten(x, 1)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.softmax(self.fc3(x))
    return x

In [89]:
net = AlexNet()
summary(net, (3, 227, 227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
          ConvPool-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
         MaxPool2d-7          [-1, 256, 13, 13]               0
          ConvPool-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
